In [1]:
import os
import scipy

from mido import MidiFile
import networkx as nx
from networkx_viewer import Viewer
import matplotlib
import matplotlib.pyplot as plt

import math
import numpy as np
import pandas as pd

from torch_geometric.data import HeteroData


print(scipy.__version__)
print(matplotlib.__version__)
print(nx.__version__)

1.7.3
3.6.2
2.8.4


In [2]:
def complete_graph(input_path) -> nx.Graph:
    edgelists = [qf for qf in os.listdir(input_path)
                 if qf.endswith('.edgelist') and not qf.startswith('_')]
    g = None

    print('loading edgelists...')
    for eg in edgelists:
        print('- ' + eg)
        h = nx.read_edgelist(os.path.join(input_path, eg), nodetype=str, create_using=nx.DiGraph(), delimiter=' ')
        for edge in h.edges():
            h[edge[0]][edge[1]]['weight'] = 1

        g = h if g is None else nx.compose(g, h)

    g = g.to_undirected()

    print('Nodes: %d' % nx.number_of_nodes(g))
    print('Edges: %d' % nx.number_of_edges(g))
    return g


In [3]:
G = complete_graph("C:\\Users\\vamvp\Desktop\Test Edgelists")

loading edgelists...
- full_edgelist.edgelist
- notes.edgelist
- program.edgelist
- tempo.edgelist
- time.signature.edgelist
Nodes: 2703
Edges: 19366


In [47]:
nodes = pd.DataFrame((list(G.nodes)), columns=['name'])
edges = pd.DataFrame(np.array(list(G.edges)), columns=['source', 'target'])

In [105]:
note_groups = [n for n in nodes['name'] if n[0] == 'g' and n[1] in [str(i) for i in range(10)] + ['-'] ]

not_group_nodes = [n for n in nodes['name'] if n not in note_groups]

url = [n for n in not_group_nodes if n[:4] == 'http']
program_nodes = []
note_nodes = []
for u in url:
    if "programs" in u:
        program_nodes.append(u)
    elif "notes" in u:
        note_nodes.append(u)
    else:
        print(u)

name_nodes = [n for n in not_group_nodes if n[0] == '-']
dur_nodes = [n for n in not_group_nodes if n[:3] == 'dur']
vel_nodes = [n for n in not_group_nodes if n[:3] == 'vel']
time_nodes = [n for n in not_group_nodes if n[:4] == 'time']
tempo_nodes = [n for n in not_group_nodes if n not in set(dur_nodes).union(vel_nodes, time_nodes, name_nodes, url)]

tempo_nodes

['11', '6', '9']

In [106]:
node_categories = {"note_group": note_groups,
                    "note": note_nodes,
                    "program": program_nodes,
                    "song_name": name_nodes,
                    "duration": dur_nodes,
                    "velocity": vel_nodes,
                    "time_sig": time_nodes,
                    "tempo": tempo_nodes
                   }

node_categories.keys()

dict_keys(['note_group', 'note', 'program', 'song_name', 'duration', 'velocity', 'time_sig', 'tempo'])

In [108]:
def add_edge_names(edges_pd: pd.DataFrame, node_cat: dict):
    edge_type = []
    augmented_edges_pd = edges_pd.copy()
    
    for i in range(len(edges.index)):
        for name in node_cat.keys():
            if edges.iloc[i]['source'] in node_cat[name]:
                edge_name_source = name + "__"
                break
        for name in node_cat.keys():
            if edges.iloc[i]['target'] in node_cat[name]:
                edge_name_target = name
                break
                
        edge_name = edge_name_source + edge_name_target
        edge_type.append(edge_name)
        
    augmented_edges_pd['edge_type'] = edge_type
    return augmented_edges_pd


In [109]:
edges_2 = add_edge_names(edges, node_categories)


In [110]:
set(edges_2['edge_type'])

{'duration__note_group',
 'note__note_group',
 'note_group__duration',
 'note_group__note',
 'note_group__song_name',
 'note_group__velocity',
 'program__song_name',
 'song_name__note_group',
 'song_name__program',
 'song_name__tempo',
 'song_name__time_sig',
 'time_sig__song_name',
 'velocity__note_group'}

In [93]:
edges_2.loc[edges_2['edge_type'] == 'song_name__note', ['source', 'target']]

,source,target
933,-Albert_King_-_Born_Under_A_Bad_Sign,http://purl.org/midi-ld/programs/30
934,-Albert_King_-_Born_Under_A_Bad_Sign,http://purl.org/midi-ld/programs/0
935,-Albert_King_-_Born_Under_A_Bad_Sign,http://purl.org/midi-ld/programs/33
9697,-B_B_King_-_How_Blue_Can_You_Get,http://purl.org/midi-ld/programs/17
9698,-B_B_King_-_How_Blue_Can_You_Get,http://purl.org/midi-ld/programs/4
9699,-B_B_King_-_How_Blue_Can_You_Get,http://purl.org/midi-ld/programs/26
9700,-B_B_King_-_How_Blue_Can_You_Get,http://purl.org/midi-ld/programs/61
9701,-B_B_King_-_How_Blue_Can_You_Get,http://purl.org/midi-ld/programs/56
9702,-B_B_King_-_How_Blue_Can_You_Get,http://purl.org/midi-ld/programs/66
9703,-B_B_King_-_How_Blue_Can_You_Get,http://purl.org/midi-ld/programs/35
